# AVL Tree

## Implementing a binary search tree first

In [345]:
    
# 30,25,35,20,15,5,10,50,60,70,65
b = AVL_tree(30)
b.add(25)
b.add(35)
b.add(20)
b.add(15)
b.add(5)
b.add(10)
b.add(50)
b.add(60)
b.add(70)
b.add(65)

print(b)

LL condition
LL condition
LR condition
RR condition
RR condition
RL condition
        5(1)
    10(2)
        15(1)
20(4)
            25(1)
        30(2)
            35(1)
    50(3)
            60(1)
        65(2)
            70(1)



In [344]:
class Tree_Node():
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        
        # for AVL tree
        self.level = 1
        
class AVL_tree():
    def __init__(self, value):
        # 紀錄 AVL tree 的 root，方便使用
        self.root = Tree_Node(value)
    
    def get_balance(self, node):
        # for recurrsive，被查詢的node如果是None，則沒有 unbalance的問題，回傳 0
        if node is None:
            return 0
        # 回傳被查詢node 的 left/right level差異
        else:
            return self.get_level(node.left) - self.get_level(node.right)
        
    def get_level(self, node):
        # 如果傳入查詢的 node (可能是 left/right) 是 None，則回傳 0
        if node is None:
            return 0
        # 如果不是 none，則回傳該node的 level
        else:
            return node.level
        
    # call recursively 
    def add(self, value, root_node=None):
        if root_node is None:
            root_node = self.root
        if value<=root_node.value:
            # 如果左側沒有 node，則加入
            if root_node.left is None:
                root_node.left = Tree_Node(value)
            # 如果左側已經有 node，則在左側以下加入node
            else:
                # 則是 AVL tree 的一個重要的地方。
                # 一般 binary tree 只要 self.add(value, root_node.left) 即可增加連結，
                # 但是 AVL tree rotation 的時候，會改變 先前 node對下一個node(new_root)的連結
                # 如果沒有 root_node.left 賦值，則無法獲得rotation後的新的連結
                # 因此， self.add()必須要 return new_root
                root_node.left = self.add(value, root_node.left)
        else:
            if root_node.right is None:
                root_node.right = Tree_Node(value)
            else:
                root_node.right = self.add(value, root_node.right)
                
        # 每被call一次，中間經過的node都要加一個level
        # 而且是 stack加回來，所以最靠近 leaf node的那個會先加
        # 然後就依序往root 方向 update level 數值，一路加回來
        # 所以需要一個 get_level function，方便處理如果 child is None 的狀況
        root_node.level = max(self.get_level(root_node.left), self.get_level(root_node.right)) +1
        
        # 加完 node level 之後，查詢是否 disbalance
        # >1 -> left>right -> left condition
        # -- root.left balance >0 / value<root.left.value -> left-left
        # -- root.left balance <0 -> left-right
        # 0,1,-1 -> balance
        # <-1 -> right condition
        # -- root.right balance >0 -> right-left
        # -- root.right balance <0 -> right-right
        balance_condition = self.get_balance(root_node)
        
        # Left-left condition
        if balance_condition>1 and value<root_node.left.value:
            print('LL condition')
            new_root = self.right_rotation(root_node)
            # 上面有解釋為什麼要 return new_noot
            return new_root
            
        # Left-right condition
        elif balance_condition>1 and value>root_node.left.value:
            print('LR condition')
            # 針對 root.left 行left rotation，回傳該rotation後的rotated_root
            rotated_root = self.left_rotation(root_node.left) 
            # 將root.left rotation後的 rotated_root，接上
            root_node.left = rotated_root
            # 再針對new_root行 right rotation
            new_root = self.right_rotation(root_node)
            return new_root
        # Right-right condition
        elif balance_condition<-1 and value>root_node.right.value:
            print('RR condition')
            new_root = self.left_rotation(root_node)
            return new_root
        # Right-left condition
        elif balance_condition<-1 and value<root_node.right.value:
            print('RL condition')
            # 針對 root.right 行 right rotation
            rotated_root = self.right_rotation(root_node.right)
            root_node.right = rotated_root
            new_root = self.left_rotation(root_node)
            return new_root
        
        # 如果都沒有 rotation，return root_node
        # 這裡的 return node 非常重要，因為要讓一個 node 知道要連接到這個 node
        return root_node


    def right_rotation(self, disbalance_node):
        new_root = disbalance_node.left  # 將原root.left設為新root
        disbalance_node.left = new_root.right # 將新 root.right搬到原root.left的空缺
        new_root.right = disbalance_node # 新root.right 設定為原root
        
        disbalance_node.level = max(self.get_level(disbalance_node.left), self.get_level(disbalance_node.right))+1
        new_root.level = max(self.get_level(new_root.left), self.get_level(new_root.right))+1
        
        # 如果 self.root 是 disbalance 的node，則要 update self.root為new_root
        if disbalance_node==self.root:
            self.root = new_root
        return new_root
    
    def left_rotation(self, disbalance_node):
        new_root = disbalance_node.right # 將原root.right設為新root
        disbalance_node.right = new_root.left
        new_root.left = disbalance_node
        
        disbalance_node.level = max(self.get_level(disbalance_node.left), self.get_level(disbalance_node.right))+1
        new_root.level = max(self.get_level(new_root.left), self.get_level(new_root.right))+1
        
        if disbalance_node==self.root:
            print(self.root.value, new_root.value)
            self.root = new_root
        return new_root
    
        
    def __str__(self, level=0, node=None):
        if node is None:
            node = self.root
        return_value = '    '*level + f"{str(node.value)}({node.level})" + '\n'
        left_value = '' if node.left is None else self.__str__(level+1, node.left)
        right_value = '' if node.right is None else self.__str__(level+1, node.right)
        return_value = left_value + return_value + right_value
        return return_value
    
    
    # traversal by using queue
    def levelOrder(self, node=None):
        if node is None:
            node = self.root
        queue = Q()
        queue.enQ(node)
        return_list = []
        
        while queue.is_empty() is False:
            # 拿出一個 node
            current_node = queue.deQ()
            return_list.append(str(f"{current_node.value}({current_node.level})"))
            # print(f"Node:{current_node.value}, Level:{self.count_level(current_node)}")
            
            # 把兩邊children 加上去
            if current_node.left is not None:
                queue.enQ(current_node.left)
            if current_node.right is not None:
                queue.enQ(current_node.right)
            
        return ' '.join(return_list)


    

    


In [ ]:
class Q():
    def __init__(self):
        self.head = None
        self.tail = None
        
    def enQ(self, value):
        new_node = N(value)
        
        if self.head is None:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node
            
    def __iter__(self):
        current_node = self.head
        while current_node:
            yield current_node
            current_node = current_node.next
            
    def __str__(self):
        ret_str = [str(n.value) for n in self if n.value is not None]
        return ' '.join(ret_str)
            
    def is_empty(self):
        if self.head is None:
            return True
        else:
            return False
    
    def deQ(self):
        if self.is_empty():
            return None
        else:
            ret_value = self.head.value
            self.head =self.head.next
            return ret_value
        
class N():
    def __init__(self, value):
        self.value = value
        self.next = None
        



## Other

In [ ]:
    # my solution
    def count_level(self, current_node=None):
        # initial call, set as self
        if current_node is None:
            current_node = self
            
        # right arm, left arm, 取高的, 加上1
        # 從最末端往回推算。leaf node -> return 0 -> 依序往上加 
        right = 0 if current_node.right is None else self.count_level(current_node.right)
        left = 0 if current_node.left is None else self.count_level(current_node.left)
        
        return max(left,right)+1
    
        

    def count_level(self, current_node=None):
        # 進行 post-order traversal，update 每一個 node 的 height/level
        if current_node is None:
            current_node = self

        # leaf node
        if current_node.left is None and current_node.right is None:
            current_node.level = 1
        # 只有 right/left 有node 的情況
        elif current_node.left is None:
            current_node.level = current_node.right.count_level()+1
        elif current_node.right is None:
            current_node.level = current_node.left.count_level() +1
        # 兩邊都有 node 的情況
        else:
            current_node.left.level = current_node.left.count_level()
            current_node.right.level = current_node.right.count_level()
            current_node.level = max(current_node.left.level,current_node.right.level) +1
        return current_node.level